In [13]:
!python multiprocess_example.py

Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\.ipynb_checkpoints
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\app_czicutter.py
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\app_czicutter05012024.py
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\app_test.py
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\environment.yml
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\libczi_cutter.ipynb
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\libczi_multiprocs.ipynb
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\libczi_multiprocs_test.ipynb
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\libczi_test.ipynb
Processing file: D:\USERS_ANALYSIS\Schatzm\GitHub\CZI_processing\libczi_notebook\multiproce

In [16]:
!python libczi_async_test.py

2024-01-11 10:03:30,048 [INFO] [SpawnPoolWorker-2] Processing file: F:\PROJECTS\Axioscan_Analysis\Vojtech_Abraham\CZI_processing\libczi\test\2023_05_22_Pyl-OB_010_small - Copy (2).czi
2024-01-11 10:03:30,093 [INFO] [SpawnPoolWorker-1] Processing file: F:\PROJECTS\Axioscan_Analysis\Vojtech_Abraham\CZI_processing\libczi\test\2023_05_22_Pyl-OB_010_small - Copy (3).czi
2024-01-11 10:03:30,102 [INFO] [SpawnPoolWorker-3] Processing file: F:\PROJECTS\Axioscan_Analysis\Vojtech_Abraham\CZI_processing\libczi\test\2023_05_22_Pyl-OB_010_small - Copy.czi
2024-01-11 10:03:30,128 [INFO] [SpawnPoolWorker-4] Processing file: F:\PROJECTS\Axioscan_Analysis\Vojtech_Abraham\CZI_processing\libczi\test\2023_05_22_Pyl-OB_010_small.czi
2024-01-11 10:03:30,242 [INFO] [SpawnPoolWorker-2] File will be divided by 1600 times 1200 pixels, in 30 tiles

  0%|          | 0/30 [00:00<?, ?it/s]2024-01-11 10:03:30,257 [INFO] [SpawnPoolWorker-1] File will be divided by 1600 times 1200 pixels, in 30 tiles

  0%|          | 

In [10]:
#app_czicutter.py
!python app_czicutter05012024.py

Processing: 2023_05_22_Pyl-OB_001
Processing: 2023_05_22_Pyl-OB_007
Processing: 2023_05_22_Pyl-OB_003
Processing: 2023_05_22_Pyl-OB_005
Processing: 2023_05_18_Pyl-OB_001
Processing: 2023_05_22_Pyl-OB_009
Processing: 2023_05_22_Pyl-OB_011
Processing: 2023_05_22_Pyl-OB_013
Processing: 2023_05_22_Pyl-OB_002
Processing: 2023_05_21_Pyl-OB_001
Processing: 2023_05_22_Pyl-OB_010
Processing: 2023_05_22_Pyl-OB_004
Processing: 2023_05_22_Pyl-OB_008
Processing: 2023_05_22_Pyl-OB_014
Processing: 2023_05_22_Pyl-OB_012
Processing: 2023_05_22_Pyl-OB_006
Processing: 2023_05_22_Pyl-OB_015
Processing: 2023_05_22_Pyl-OB_017
Processing: 2023_05_22_Pyl-OB_019
Processing: 2023_05_22_Pyl-OB_021
Processing: 2023_05_22_Pyl-OB_023
Processing: 2023_05_22_Pyl-OB_025
Processing: 2023_05_22_Pyl-OB_027
Processing: 2023_05_22_Pyl-OB_029
Processing: 2023_05_22_Pyl-OB_016
Processing: 2023_05_22_Pyl-OB_018
Processing: 2023_05_22_Pyl-OB_022
Processing: 2023_05_22_Pyl-OB_020
Processing: 2023_05_22_Pyl-OB_026
Processing: 20


100%|##########| 80/80 [1:08:41<00:00, 51.52s/it]

100%|##########| 935/935 [9:47:59<00:00, 37.73s/it]

100%|##########| 750/750 [10:02:44<00:00, 48.22s/it]

100%|##########| 750/750 [10:20:29<00:00, 49.64s/it]

100%|##########| 750/750 [10:27:28<00:00, 50.20s/it]

100%|##########| 750/750 [10:29:10<00:00, 50.33s/it]

100%|##########| 750/750 [10:33:36<00:00, 50.69s/it]

100%|##########| 750/750 [10:40:44<00:00, 51.26s/it]

100%|##########| 750/750 [10:14:26<00:00, 49.16s/it]

 31%|###       | 271/880 [2:10:27<4:53:10, 28.88s/it]

100%|##########| 750/750 [10:11:10<00:00, 48.89s/it]

100%|##########| 750/750 [10:14:33<00:00, 49.16s/it]

100%|##########| 750/750 [10:24:47<00:00, 49.98s/it]

100%|##########| 750/750 [10:23:54<00:00, 49.91s/it]

100%|##########| 750/750 [10:30:33<00:00, 50.44s/it]

100%|##########| 750/750 [10:28:24<00:00, 50.27s/it]

100%|##########| 750/750 [10:06:44<00:00, 48.54s/it]

100%|##########| 750/750 [9:44:55<00:00, 46.79s/it]

100%|##########| 725/725 [9:47:

In [ ]:
xxx

In [1]:
import tifffile
from multiprocessing import Pool
from pylibCZIrw import czi as pyczi
import numpy as np
import os, sys
from tqdm import tqdm
from tqdm.contrib import itertools as it


In [ ]:
class FileProcessor:
    def __init__(self, folder_path, output_folder, num_processes=4, roi_width=512, roi_height=512):
        self.folder_path = folder_path
        self.output_folder = output_folder
        self.num_processes = num_processes
        self.roi_width = roi_width
        self.roi_height = roi_height

    def libczi_cutter(self, czifile, name):
        if not os.path.exists(self.output_folder + "/" + name):
            os.makedirs(self.output_folder + "/" + name)

        from cztile.fixed_total_area_strategy import AlmostEqualBorderFixedTotalAreaStrategy2D

        tiler = AlmostEqualBorderFixedTotalAreaStrategy2D(
            total_tile_width=self.roi_width,
            total_tile_height=self.roi_height,
            min_border_width=0
        )

        
        with pyczi.open_czi(czifile) as czidoc_r:
            total_bounding_box = czidoc_r.total_bounding_box
            md_xml = czidoc_r.metadata

            tiles = tiler.tile_rectangle(czidoc_r.scenes_bounding_rectangle[0])
            tile_counter = 0
            for tile in tqdm(tiles):
                BGR_stack = np.zeros([total_bounding_box['Z'][1], 3, self.roi_height, self.roi_width])

                for z in range(0, total_bounding_box['Z'][1]):
                    tile2d = czidoc_r.read(
                        plane={"C": 0, "Z": z, "T": 0},
                        roi=(
                            tile.roi.x,
                            tile.roi.y,
                            tile.roi.w,
                            tile.roi.h
                        )
                    )

                    BGR_stack[z, :, :, :] = np.transpose(tile2d, (2, 0, 1))

                RGB_stack = np.zeros([total_bounding_box['Z'][1], 3, self.roi_height, self.roi_width])
                RGB_stack[:, 0, :, :] = BGR_stack[:, 2, :, :]
                RGB_stack[:, 1, :, :] = BGR_stack[:, 1, :, :]
                RGB_stack[:, 2, :, :] = BGR_stack[:, 0, :, :]

                img_name = name + '_X-{}_Y-{}_stack-{}.tif'.format(tile.roi.x, tile.roi.y, tile_counter)

                output_file = os.path.join(self.output_folder + "/" + name, img_name)
                tifffile.imwrite(output_file, RGB_stack.astype('uint8'), imagej=False, photometric='rgb',
                                 metadata=md_xml)
                tile_counter = tile_counter + 1

    def process_file(self, file_path):
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        self.libczi_cutter(file_path, file_name)

    def process_folder(self):
        files = [os.path.join(self.folder_path, file) for file in os.listdir(self.folder_path)]

        with Pool(self.num_processes) as pool:
            print(f'Processing: {czifile}')
            pool.map(self.process_file, files)

    def setup_logging(self):
        log_format = '%(asctime)s [%(levelname)s] [%(processName)s] %(message)s'
        logging.basicConfig(level=logging.INFO, format=log_format)

if __name__ == "__main__":
    # Example usage
    folder_path = "F:\\PROJECTS\\Axioscan_Analysis\\Vojtech_Abraham\\CZI_processing\\libczi\\test"
    output_folder = "F:\\PROJECTS\\Axioscan_Analysis\\Vojtech_Abraham\\CZI_processing\\libczi\\out"
    num_processes = 4

    file_processor = FileProcessor(folder_path, output_folder, num_processes, roi_width=1920, roi_height=1600)
    file_processor.setup_logging()  # Setup logging for multiprocessing
    file_processor.process_folder()
